In [1]:
import requests
from bs4 import BeautifulSoup
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import pandas as pd
from lxml.html import parse

In [13]:
def wiki_age_scrape(names):
    age = ''
    for name in names:
        if name in ages.keys():
            print "%s already in dictionary!" %name
        else:
            print "getting info for %s...." %name
            chromedriver = "/Applications/chromedriver"
            os.environ["webdriver.chrome.driver"] = chromedriver
            driver = webdriver.Chrome(chromedriver)
            driver.get("https://www.wikipedia.org/")
            e = driver.find_element_by_name('search')
            e.send_keys(name)
            e.send_keys(Keys.RETURN)
            soup = BeautifulSoup(driver.page_source)
            try:
                bday = soup.find("span", {"class": "bday"}).text.strip().encode('ascii', 'ignore')
                print "%s: %s" % (name, bday)
                ages[name] = bday
                driver.close()
            except:
                print "no age information for %s!" %name
    return ages

In [2]:
def get_salary_imdb(names):
    for name in names:
        movies = []
        salaries = []
        print name
        filename = "/Users/kpully/ds/metisgit/nyc16_ds8/projects/02-luther/csvs/%s salary.csv" %name
        if name in names_none:
            print 'already checked, no salary info!'
            print "_____________________________________"
        elif os.path.isfile(filename):
            print "%s already exists!" %filename
            print "_____________________________________"
        else:
            print "Getting information for %s....." %name
            chromedriver = "/Applications/chromedriver"
            os.environ["webdriver.chrome.driver"] = chromedriver
            driver = webdriver.Chrome(chromedriver)
            driver.get("http://www.imdb.com/?ref_=nv_home")
            SEARCH_BUTTON_XPATH = '//input[@type="text" and @id="navbar-query"]'
            e = driver.find_element_by_xpath(SEARCH_BUTTON_XPATH)
            e.send_keys(name)
            e.send_keys(Keys.RETURN)
            
            html = driver.page_source
            soup = BeautifulSoup(html)

            td = soup.find('td', {"class": "result_text"})
            url = td.find('a').get('href')
            url = 'http://www.imdb.com' + url
            driver.get(url) 
            try:
                bio = driver.find_element_by_link_text('Biography')
                bio.click()
            except:
                print "no biography exists!"
                names_none.add(name)
                print "_____________________________________"
            else:
                try:
                    salary = driver.find_element_by_link_text('Salary')
                    salary.click()
                except:
                    print "no salary information available!"
                    names_none.add(name)
                    print "_____________________________________"
                else:
                    #html = driver.page_source
                    #soup = BeautifulSoup(html, "lxml")
                    url = driver.current_url
                    page = parse(url)
                    xp = '//table[@id="salariesTable"]'
                    rows = page.xpath(xp)
                    rows = rows[0].text_content().replace('\n', '').strip().encode('ascii', 'ignore')

                    data = []
                    for row in rows.strip().split('     '):
                        data.append(row.strip())
                    data = filter(None, data)
                    movies = data[::2]
                    print movies
                    salaries = data[1::2]
                    print salaries
                    df = pd.DataFrame({'movie': movies, 'salary': salaries})
                    df.to_csv('/Users/kpully/ds/metisgit/nyc16_ds8/projects/02-luther/csvs/%s salary.csv' %name, encoding='utf-8')
                    print "_____________________________________"

            driver.close()

In [3]:
def get_url_imdb(name):
    chromedriver = "/Applications/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    driver.get("http://www.imdb.com/?ref_=nv_home")
    print name
    SEARCH_BUTTON_XPATH = '//input[@type="text" and @id="navbar-query"]'
    e = driver.find_element_by_xpath(SEARCH_BUTTON_XPATH)
    e.send_keys(name)
    e.send_keys(Keys.RETURN)
    html = driver.page_source
    soup = BeautifulSoup(html)

    td = soup.find('td', {"class": "result_text"})
    url = td.find('a').get('href')
    url = 'http://www.imdb.com' + url

    driver.close()
    return url

In [4]:
def get_filmography_imdb(name, url): 
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    movies = []
    years = []
    for film in soup.find_all('div',{ "class" : "filmo-row" }):
        #print film
        year = film.find('span', {"class": "year_column"}).text.strip()
        years.append(year)
        movie = film.find('a').text
        movies.append(movie)
        df = pd.DataFrame({'movie' : movies, 'year' : years})
        df.to_csv('%s.csv' %name, encoding='utf-8')

In [9]:
def get_url_list_imdb(names):
    for name in names:
        filename = "/Users/kpully/ds/metisgit/nyc16_ds8/projects/02-luther/csvs/%s.csv" %name
        filename_other = "/Users/kpully/ds/metisgit/nyc16_ds8/projects/02-luther/%s.csv" %name
        if os.path.isfile(filename):
            print "%s already exists!" %filename
        elif os.path.isfile(filename_other):
            print "%s already exists!" %filename_other
        else:
            chromedriver = "/Applications/chromedriver"
            os.environ["webdriver.chrome.driver"] = chromedriver
            driver = webdriver.Chrome(chromedriver)
            driver.get("http://www.imdb.com/?ref_=nv_home")
            print name
            SEARCH_BUTTON_XPATH = '//input[@type="text" and @id="navbar-query"]'
            e = driver.find_element_by_xpath(SEARCH_BUTTON_XPATH)
            e.send_keys(name)
            e.send_keys(Keys.RETURN)
            html = driver.page_source
            soup = BeautifulSoup(html)

            td = soup.find('td', {"class": "result_text"})
            url = td.find('a').get('href')
            url = 'http://www.imdb.com' + url

            driver.close()

            get_filmography(name, url)

In [4]:
def get_sex(names):
    for name in names:
        if name in sexes.keys():
            print "%s already in dict!" %name
        else:
            chromedriver = "/Applications/chromedriver"
            os.environ["webdriver.chrome.driver"] = chromedriver
            driver = webdriver.Chrome(chromedriver)
            driver.get("http://www.imdb.com/?ref_=nv_home")

            SEARCH_BUTTON_XPATH = '//input[@type="text" and @id="navbar-query"]'
            e = driver.find_element_by_xpath(SEARCH_BUTTON_XPATH)
            e.send_keys(name)
            e.send_keys(Keys.RETURN)
            html = driver.page_source
            soup = BeautifulSoup(html)

            td = soup.find('td', {"class": "result_text"})
            url = td.find('a').get('href')
            url = 'http://www.imdb.com' + url

            driver.get(url)
            html = driver.page_source
            soup = BeautifulSoup(html)
            try:
                sex = soup.find('span', {'itemprop': 'jobTitle'}).text.strip()
            except: 
                print "can't find sex info!"
            sexes[name] = sex

In [5]:
import fnmatch